In [1]:
import numpy as np
import tnn
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import logging

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

In [2]:
dataset = load_dataset("stanfordnlp/imdb")
train = dataset.get("train")
test = dataset.get("test")

In [3]:
num_labels = len(train.unique("label"))
name = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(name)

In [4]:
def pre_process(examples):
    examples = tokenizer(
        examples["text"],
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    return examples


train_dataset = train.map(
    pre_process,
    batched=True,
    num_proc=2,
)
test_dataset = test.map(
    pre_process,
    batched=True,
    num_proc=2,
)

In [5]:
def collate_fn(batch):
    inputs = {"input_ids": [], "attention_mask": []}
    labels = []
    for example in batch:
        inputs["input_ids"].append(example["input_ids"])
        inputs["attention_mask"].append(example["attention_mask"])
        labels.append(example["label"])

    inputs["input_ids"] = torch.tensor(inputs["input_ids"]).long()
    inputs["attention_mask"] = torch.tensor(inputs["attention_mask"]).long()
    labels = torch.tensor(labels).long()
    return inputs, labels

In [6]:
trainloader = data.DataLoader(
    train_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    drop_last=False,
    shuffle=True,
    pin_memory=True,
    num_workers=2,
)

testloader = data.DataLoader(
    test_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    drop_last=False,
    shuffle=True,
    pin_memory=True,
    num_workers=2,
)


def to(inputs, labels, device, non_blocking):
    inputs = dict(
        map(
            lambda item: (item[0], item[1].to(device, non_blocking=non_blocking)),
            inputs.items(),
        )
    )
    labels = labels.to(device, non_blocking=True)
    return inputs, labels

In [7]:
lr = 5e-5
weight_decay = 1e-4

model = AutoModelForSequenceClassification.from_pretrained(name, num_labels=num_labels)
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss()

In [8]:
trainer = tnn.Trainer(
    model,
    optimizer,
    loss_fn,
    trainloader,
    testloader,
    epochs=3,
    store_update_metrics=True,
    unpack_inputs=True,
    device=device,
    to_fn=to,
    path="../training/imdb-distil-bert-base.h5",
    verbose=True,
    profile=True,
    logger_name="base",
)

In [9]:
base_metrics = trainer.train()

2024-12-18 19:40:21,772 - INFO - model using cuda
2024-12-18 19:40:26,072 - INFO - training started
2024-12-18 19:44:18,619 - INFO - (epoch: 1/3): (train loss: 0.6907, test loss: 0.6884, train acc: 51.97%, test acc: 52.03%)
(gpu memory profile): (average allocated: 545.0 MB, average reserved: 5597.0 MB)
(duration info): (epoch duration: 0:03:52, elapsed time: 0:03:52)
2024-12-18 19:48:07,079 - INFO - (epoch: 2/3): (train loss: 0.6879, test loss: 0.6862, train acc: 55.52%, test acc: 61.02%)
(gpu memory profile): (average allocated: 545.0 MB, average reserved: 5597.0 MB)
(duration info): (epoch duration: 0:03:48, elapsed time: 0:07:41)
2024-12-18 19:51:58,879 - INFO - (epoch: 3/3): (train loss: 0.6859, test loss: 0.6840, train acc: 57.87%, test acc: 67.58%)
(gpu memory profile): (average allocated: 545.0 MB, average reserved: 5597.0 MB)
(duration info): (epoch duration: 0:03:51, elapsed time: 0:11:32)
2024-12-18 19:51:58,879 - INFO - training complete
2024-12-18 19:51:58,891 - INFO - tra